In [2]:
from matplotlib import pyplot as plt
from tifffile import tifffile

"""full 1 channel tiff writer without dataframe log"""



def create_marked_file_final(xml_file, input_file, output_file):
    import numpy as np
    import tifffile
    import xml.etree.ElementTree as ET

    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    print(original_image.shape)
    binary_image = np.zeros((layers, 1, height, width), dtype=np.uint8)
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for marker_type in root.findall('Marker_Data/Marker_Type'):
        type_id = int(marker_type.find('Type').text)

        for marker in marker_type.findall('Marker'):

            x = int(marker.find('MarkerX').text) -1
            y = int(marker.find('MarkerY').text) -1
            z_raw = ((int(marker.find('MarkerZ').text) + 2 ) // 3) -1

            binary_image[z_raw, 0, y, x] = 255

    tifffile.imwrite(output_file, binary_image)
    return binary_image


create_marked_file_final('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/FF151L2L_marked7.tif')



(56, 3, 512, 512)


array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       ...,


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 

In [1]:
"""Full dataframe logger from xml file"""



def pull_marker_data(xml_file):
    import xml.etree.ElementTree as et
    import pandas as pd
    file = et.parse(xml_file)
    root = file.getroot()
    marker_data = []

    MDElement = root.find('Marker_Data')
    for Marker_Type in MDElement.findall('Marker_Type'):
        type_id = int(Marker_Type.find('Type').text)

        for Marker in Marker_Type.findall('Marker'):
            xloc = int(Marker.find('MarkerX').text) if Marker.find('MarkerX') is not None else None
            yloc = int(Marker.find('MarkerY').text) if Marker.find('MarkerY') is not None else None
            zloc = int(Marker.find('MarkerZ').text) if Marker.find('MarkerZ') is not None else None
            zlayer = (2 + zloc) // 3

            marker_data.append({
                'z': zlayer,
                'type_id': type_id,
                'y': yloc,
                'x': xloc,
            })

    dataframe = pd.DataFrame(marker_data, columns=['z', 'type_id', 'y', 'x'])
    return dataframe

result = pull_marker_data('FF15A/CellCounter_FF151L2L.xml')
result.head(96)

FileNotFoundError: [Errno 2] No such file or directory: 'FF15A/CellCounter_FF151L2L.xml'

In [37]:
result = pull_marker_data(r'C:\Users\richa\parralabimageprocessing\Synapse Confocal Images Rat\BATA_\CellCounter_BAT1L2L.xml')
result.head(96)

,z,type_id,y,x
0,2,1,281,267
1,2,1,285,273
2,2,1,287,278
3,2,1,271,288
4,2,1,277,292
5,2,1,285,291
6,2,2,181,160
7,2,2,184,166
8,2,2,183,169
9,2,2,160,163


In [28]:
"""full 1 channel tiff writer from dataframe log"""


def pull_synapse_and_mark_file(xml_file, input_file, output_file):
    import numpy as np
    import tifffile

    df = pull_marker_data(xml_file)

    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    binary_image = np.zeros((layers, 1, height, width), dtype=np.uint8)

    x = df['x']
    y = df['y']
    z = df['z']

    for i in range(len(df)):
        binary_image[z[i] - 1,:, y[i] - 1, x[i] - 1] = 255

    tifffile.imwrite(output_file, binary_image)
    return binary_image

# Example usage
pull_synapse_and_mark_file('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/FF151L2L_marked93.tif')

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       ...,


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 

In [2]:
"""full 3 channel tiff writer without dataframe log"""

def create_colored_file_final(xml_file, input_file, output_file):
    import numpy as np
    import tifffile
    import xml.etree.ElementTree as ET

    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    print(original_image.shape)
    rgb_image = np.zeros((layers, 3, height, width), dtype=np.uint8)
    tree = ET.parse(xml_file)
    root = tree.getroot()
    typecount = 0
    for marker_type in root.findall('Marker_Data/Marker_Type'):
        typecount += 1
        continue
    type_array = []
    start_color = np.array([255, 0, 0])
    end_color = np.array([0, 0, 255])
    for i in range(typecount):
        color = (start_color + (end_color - start_color) * i / (typecount - 1)).astype(int)
        type_array.append(tuple(color))
    for marker_type in root.findall('Marker_Data/Marker_Type'):
        type_id = int(marker_type.find('Type').text)
        if type_id <= typecount // 2:
            color = (
                np.array([255, 0, 0]) + (np.array([0, 255, 0]) - np.array([255, 0, 0])) * type_id / (typecount - 1)
            ).astype(int)
        else:
            color = (
                np.array([0, 255, 0]) + (np.array([0, 0, 255]) - np.array([0, 255, 0])) * (type_id - (typecount // 2)) / (typecount - 1)
            ).astype(int)

        for marker in marker_type.findall('Marker'):

            x = int(marker.find('MarkerX').text) - 1
            y = int(marker.find('MarkerY').text) - 1
            z = ((int(marker.find('MarkerZ').text) + 2) // 3) - 1

            rgb_image[z, :, y, x] = color  # Apply color to all three channels for the specific pixel

    tifffile.imwrite(output_file, rgb_image)
    return rgb_image


create_colored_file_final('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/FF151L2L_marked72.tif')

(56, 3, 512, 512)


array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
  

In [23]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_3d_nonzero_pixels(image):
    z, c, y, x


SyntaxError: unterminated triple-quoted string literal (detected at line 19) (1714432801.py, line 19)

In [2]:
""" create the most distinct colors from the dataframe"""



def find_distinct_colors(xml_file):
    import numpy as np
    import colorsys

    df = pull_marker_data(xml_file)
    num_colors = max(df['type_id'])

    colors = []
    for i in range(num_colors):
        r, g, b = colorsys.hsv_to_rgb(i / num_colors, 1.0, 1.0)
        rgb = np.array([int(r * 255), int(g * 255), int(b * 255)])
        colors.append(rgb)

    return colors


print(find_distinct_colors('FF15A/CellCounter_BAT1L2L.xml'))




FileNotFoundError: [Errno 2] No such file or directory: 'FF15A/CellCounter_FF151L2L.xml'

In [18]:
"""full 3 channel hue color tiff writer from dataframe log"""

"full working code readable on napari image viewer"

def pull_synapse_and_mark_file(xml_file, input_file, output_file):
    import numpy as np
    import tifffile
    df = pull_marker_data(xml_file)
    color_array = find_distinct_colors(xml_file)
    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    rgb_image = np.zeros((layers, 3, height, width), dtype=np.uint8)
    x = df['x']
    y = df['y']
    z = df['z']
    type_id = df['type_id']
    for i in range(len(df)):
        rgb_image[z[i], :, y[i], x[i]] = color_array[type_id[i]-1]
    tifffile.imwrite(output_file, rgb_image)
    return rgb_image

pull_synapse_and_mark_file('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/FF151L2L_marked94.tif')




array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
  

In [18]:
"image marking reader"

import tifffile
import numpy as np  # Make sure numpy is imported

image = tifffile.imread('FF15/FF151L2L_marked92.tif')
# og = tifffile.imread('FF15/FF151L2L.lsm') # This line is not strictly needed for the current request

print("Image shape:", image.shape)
# print("Original LSM shape:", og.shape) # Not strictly needed for the current request

# Find the indices of non-zero elements
indices_of_non_zero = np.nonzero(image)

# Extract z, y, x coordinates based on the image shape
# z corresponds to layers (dimension 0)
# y corresponds to height (dimension 2)
# x corresponds to width (dimension 3)
z_coords = indices_of_non_zero[0]
y_coords = indices_of_non_zero[2]
x_coords = indices_of_non_zero[3]

if z_coords.size > 0:  # Check if there are any non-zero values
    # Stack the spatial coordinates: each row becomes [z, y, x]
    all_hit_spatial_coords = np.stack((z_coords, y_coords, x_coords), axis=1)

    # Find unique spatial coordinates (unique rows of [z, y, x])
    unique_spatial_coordinates = np.unique(all_hit_spatial_coords, axis=0)

    print(
        f"\nFound {len(unique_spatial_coordinates)} unique (z, y, x) spatial locations with non-zero values in the 'image':")

    # Print all unique coordinates directly
    for coord in unique_spatial_coordinates:
        print(f"z: {coord[0]}, y: {coord[1]}, x: {coord[2]}")
else:
    print("\nNo non-zero values found in the 'image'.")


Image shape: (56, 3, 512, 512)

Found 96 unique (z, y, x) spatial locations with non-zero values in the 'image':
z: 10, y: 58, x: 431
z: 10, y: 61, x: 410
z: 10, y: 67, x: 428
z: 10, y: 69, x: 426
z: 10, y: 76, x: 424
z: 10, y: 85, x: 395
z: 10, y: 92, x: 397
z: 10, y: 92, x: 414
z: 10, y: 93, x: 411
z: 10, y: 182, x: 108
z: 10, y: 189, x: 101
z: 10, y: 192, x: 98
z: 10, y: 192, x: 112
z: 10, y: 199, x: 44
z: 10, y: 199, x: 112
z: 10, y: 202, x: 47
z: 10, y: 205, x: 79
z: 10, y: 205, x: 114
z: 10, y: 207, x: 48
z: 10, y: 210, x: 29
z: 10, y: 212, x: 113
z: 10, y: 218, x: 18
z: 10, y: 222, x: 79
z: 10, y: 225, x: 81
z: 10, y: 228, x: 98
z: 10, y: 230, x: 31
z: 10, y: 238, x: 27
z: 22, y: 446, x: 32
z: 22, y: 452, x: 27
z: 22, y: 452, x: 32
z: 22, y: 455, x: 36
z: 22, y: 460, x: 358
z: 22, y: 462, x: 34
z: 22, y: 464, x: 6
z: 22, y: 468, x: 29
z: 22, y: 469, x: 6
z: 22, y: 471, x: 375
z: 22, y: 473, x: 25
z: 22, y: 475, x: 345
z: 22, y: 479, x: 372
z: 22, y: 480, x: 345
z: 22, y: 484, x:

In [27]:
"""full 3 channel hue color 2d image folder writer from dataframe log"""


def create_rgb_image_folder(xml_file, input_file, output_folder):
    """Save each layer as individual RGB image instead of a single TIFF"""
    import numpy as np
    import tifffile
    import xml.etree.ElementTree as ET
    import os
    from PIL import Image

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Load original image and get dimensions
    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    print(f"Image shape: {original_image.shape}")

    # Create a binary mask for marked points
    binary_mask = np.zeros((layers, height, width), dtype=bool)

    # Parse XML to get marker coordinates
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Extract marker positions from XML
    for marker_type in root.findall('Marker_Data/Marker_Type'):
        for marker in marker_type.findall('Marker'):
            x = int(marker.find('MarkerX').text) - 1
            y = int(marker.find('MarkerY').text) - 1
            z_raw = ((int(marker.find('MarkerZ').text) + 2) // 3) - 1

            binary_mask[z_raw, y, x] = True

    # Save each layer as an RGB image
    for z in range(layers):
        # Create RGB image (height, width, 3)
        rgb_layer = np.zeros((height, width, 3), dtype=np.uint8)

        # Copy original image colors where marked
        for y in range(height):
            for x in range(width):
                if binary_mask[z, y, x]:
                    # Convert from (channel, y, x) to (y, x, channel) format
                    rgb_layer[y, x, :] = original_image[z, :, y, x]

        # Save the image
        img = Image.fromarray(rgb_layer)
        img.save(f"{output_folder}/layer_{z:03d}.png")

    return f"Saved {layers} images to {output_folder}"


# Example usage
create_rgb_image_folder('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/rgb_layers')




Image shape: (56, 3, 512, 512)


'Saved 56 images to FF15/rgb_layers'

In [32]:
"""automated image marker from folder"""

def mark_folder(file_folder):
    import os
    import numpy as np
    import tifffile

    folder = file_folder
    output = folder + "_markers"
    os.makedirs(output, exist_ok=True)
    lsm_list = [file for file in os.listdir(folder) if file.endswith(".lsm") and not (file.startswith(".") or file.endswith("old.lsm"))]
    if len(lsm_list) == 0:
        print("No .lsm files found in the folder.")
        return
    else:
        for file in lsm_list:
            print(file)
            image = tifffile.imread(folder + "/" + file)
            layers, channels, height, width = image.shape
            rgb_image = np.zeros((layers, channels, height, width), dtype=np.uint8)
            xml_file_path = folder + "A/CellCounter_" + file.replace(".lsm", ".xml")
            xml_alternate_path = folder + "A_/CellCounter_" + file.replace(".lsm", ".xml")
            xml_window_path = folder + "A/CellCounter_Counter Window - " + file.replace(".lsm", ".xml")
            if os.path.exists(xml_file_path):
                df = pull_marker_data(xml_file_path)
                color_array = find_distinct_colors(xml_file_path)
            else:
                if os.path.exists(xml_alternate_path):
                    df = pull_marker_data(xml_alternate_path)
                    color_array = find_distinct_colors(xml_alternate_path)
                else:
                    if os.path.exists(xml_window_path):
                        df = pull_marker_data(xml_window_path)
                        color_array = find_distinct_colors(xml_window_path)
                    else:
                        if os.path.exists(folder + "Counter Window - " + file.replace(".lsm", ".xml")):

                            df = pull_marker_data(folder + "Counter Window - " + file.replace(".lsm", ".xml"))
                            color_array = find_distinct_colors(folder + "Counter Window - " + file.replace(".lsm", ".xml"))

                        else:
                            print(xml_file_path + "does not exist. Skipping file.")

            x = df['x']
            y = df['y']
            z = df['z']
            type_id = df['type_id']
            filename = file.replace(".lsm", "_marked.tif")
            filepath = os.path.join(output, filename)
            for i in range(len(df)):
                rgb_image[z[i]-1, :, y[i]-1, x[i]-1] = color_array[type_id[i]-1]
            tifffile.imwrite(filepath, rgb_image)


In [33]:
import os
path = r"C:\Users\richa\parralabimageprocessing\Synapse Confocal Images Rat"
for folder in os.listdir(path):
    if not (folder.startswith(".") or folder.startswith("w")):
        mark_folder(os.path.join(path, folder))


BAT1L2L.lsm
BAT1L2R.lsm
BAT1L5L.lsm
BAT1L5R.lsm
BAT2L2L.lsm
BAT2L2R.lsm
BAT2L5L.lsm
BAT2L5R.lsm
No .lsm files found in the folder.
FF151L2L.lsm
FF151L2R.lsm
FF151L5L.lsm
FF151L5R.lsm
FF152L2L.lsm
FF152L2R.lsm
FF152L5L.lsm
FF152L5R.lsm
No .lsm files found in the folder.
FF171L2L.lsm
FF171L2R.lsm
FF171L5L.lsm
FF171L5R.lsm
FF172L2L.lsm
FF172L2R.lsm
FF172L5L.lsm
FF172L5R.lsm
No .lsm files found in the folder.
FF231L2L.lsm
FF231L2R.lsm
FF231L5L.lsm
FF231L5R.lsm
FF232L2L.lsm
FF232L2R.lsm
FF232L5L.lsm
FF232L5R.lsm
No .lsm files found in the folder.
OBFF091L2L.lsm
OBFF091L2R.lsm
OBFF091L5L.lsm
OBFF091L5R.lsm
OBFF092L2L.lsm
OBFF092L2R.lsm
OBFF092L5L.lsm
OBFF092L5R.lsm
No .lsm files found in the folder.
OBFF11L2L.lsm
OBFF11L2R.lsm
OBFF11L5L.lsm
OBFF11L5R.lsm
OBFF12L2L.lsm
OBFF12L2R.lsm
OBFF12L5L.lsm
OBFF12L5R.lsm
OBFF101L2L.lsm
OBFF101L2R.lsm
OBFF101L5L.lsm
OBFF101L5R.lsm
OBFF102L2L.lsm
OBFF102L2R.lsm
OBFF102L5L.lsm
OBFF102L5R.lsm
No .lsm files found in the folder.
OBFF111L2L.lsm
OBFF111L2R.lsm
